# NEWS RECOMMENDER


Some of the outputs and visualizations used in the report might not be produced while running the code.

### SCRAPING DATA



Uncomment and run the below cell to scrape data from BBC.com. The csv file keeps appending after every run. Otherwise, load a downloaded data set.

In [ ]:

# from nltk.corpus import stopwords
# from nltk.tokenize import TweetTokenizer
# from nltk.stem.snowball import SnowballStemmer
# import re
# import pickle
# import pandas as pd
# from bs4 import BeautifulSoup
# import requests
# import csv

# from newspaper import Article 

# url_base = "https://bbc.com"
# sub_url = ['/news/','/sports/','/reel/','/worklife/','/travel/','/future/']
# sub_url = [url_base + i for i in sub_url]

# links = []

# for url in sub_url:
#     response = requests.get(url)
#     data = response.text
#     soup = BeautifulSoup(data,'lxml')
#     tags = soup.find_all('a')
#     for tag in tags:
#         links.append(tag.get('href'))

# new_list=[]
# for i in links:
#     if '/news/' in i:
#         new_list.append(i)
#     if '/sports/' in i:
#         new_list.append(i)
#     if '/reel/' in i:
#         new_list.append(i)
#     if '/worklife/' in i:
#         new_list.append(i)
#     if '/travel/' in i:
#         new_list.append(i)
#     if '/future/' in i:
#         new_list.append(i)

# newlist = []
# for i in new_list:
#     if i[0] == '/':
#         continue
#     else:
#         newlist.append(i)
#         new_list.remove(i)
# newlist = [url_base + i for i in new_list]
# for i  in newlist:
#     if 'comhttps' in i:
#         newlist.remove(i)

# articles = []
# for url in newlist:
#     article = Article(url)
#     try:    
#         articles.append(article)
#         article.download()
#         article.parse()
#         article.nlp()
#     except:
#         pass

# df = pd.DataFrame()
# article_link = []
# article_title = []
# article_text = []
# article_author = []
# article_keywords = []
# article_summary = []
# for article in articles:
#     article_link.append(article.url)
#     article_author.append(article.authors)
#     article_keywords.append(article.keywords)
#     article_summary.append(article.summary)
#     article_text.append(article.text)
#     article_title.append(article.title)
    
# df['Title'] = article_title
# df['Link'] = article_link
# df['Authors'] = article_author
# df['Article Text'] = article_text
# df['Summary'] = article_summary
# df['Keywords'] = article_keywords
# df.to_csv('BBC_scraped.csv',mode="a",header=False)
# datafile = 'BBC_scraped.csv'



Run this cell to load an existing file

In [ ]:
datafile = 'C:/Users/quiz4/Downloads/NewsArticles.csv'


### Loading the Dataset

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display
from tqdm import tqdm
from collections import Counter
import ast

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sb

from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob
import scipy.stats as stats

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
output_notebook()
import random
%matplotlib inline


raw_data = pd.read_csv(datafile, parse_dates=[0], infer_datetime_format=True,encoding='latin1')
raw_data=raw_data.dropna(subset=['Article Text'])  
reindexed_data = raw_data['Article Text']
reindexed_data.index = raw_data['Link']
article_length=[]
for i in range(len(reindexed_data)):
    article_length.append(len(reindexed_data[i].split()))



In [ ]:
clean_data=raw_data[['Link','Title','Article Text']]
clean_data.head()

### Cleaning, Stemming, and Removing Stop Words

In [ ]:
#Expanding Stop Word list
my_additional_stop_word_list = ['said','people','says','told','from']
from sklearn.feature_extraction import text 
stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_word_list)

from nltk.tokenize import sent_tokenize, word_tokenize
import re
def sent_to_words(sentences): #This function removes unnecessary characters, stop words, and short words, and lowercases the text 
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  
        sent = re.sub('\s+', ' ', sent)  
        sent = re.sub("\'", "", sent)
        sent= re.sub('[^\w_\s-]',' ', sent)
        sent = word_tokenize(sent)
        sent = [w.lower() for w in sent if not w.lower() in stop_words]
        sent = [i for i in sent if i.isalpha() and len(i)>2]   
        yield(sent)  
        
        
import nltk 
from nltk.stem import PorterStemmer

word_stemmer = PorterStemmer()

data = clean_data['Article Text'].values.tolist()
data_words = list(sent_to_words(data))

for i in range(len(data_words)):
    for j in range(len(data_words[i])):
        data_words[i][j]= word_stemmer.stem(data_words[i][j])

detokenized_doc = []
for i in range(len(clean_data)):
    t = ' '.join(data_words[i])
    detokenized_doc.append(t)
clean_data['cleaned_text']=detokenized_doc 
clean_data.head()

In [ ]:
reindexed_data= clean_data['cleaned_text']

### Some Exploratory Data Analysis

In [ ]:
# Define helper functions
def get_top_n_words(n_top_words, count_vectorizer, text_data):
    '''
    returns a tuple of the top n words in a sample and their 
    accompanying counts, given a CountVectorizer object and text sample
    '''
    vectorized_headlines = count_vectorizer.fit_transform(text_data.values)
    vectorized_total = np.sum(vectorized_headlines, axis=0)
    word_indices = np.flip(np.argsort(vectorized_total)[0,:], 1)
    word_values = np.flip(np.sort(vectorized_total)[0,:],1)
    
    word_vectors = np.zeros((n_top_words, vectorized_headlines.shape[1]))
    for i in range(n_top_words):
        word_vectors[i,word_indices[0,i]] = 1

    words = [word[0].encode('ascii').decode('utf-8') for 
             word in count_vectorizer.inverse_transform(word_vectors)]

    return (words, word_values[0,:n_top_words].tolist()[0])

In [ ]:
count_vectorizer = CountVectorizer(stop_words=stop_words)
words, word_values = get_top_n_words(n_top_words=15,
                                     count_vectorizer=count_vectorizer, 
                                     text_data=reindexed_data)

fig, ax = plt.subplots(figsize=(16,8))
ax.bar(range(len(words)), word_values);
ax.set_xticks(range(len(words)));
ax.set_xticklabels(words, rotation='vertical');
ax.set_title('Top words in articles dataset (excluding stop words)');
ax.set_xlabel('Word');
ax.set_ylabel('Number of occurences');
plt.show()

### Feature Space Construction

In [ ]:
count_vectorizer = CountVectorizer(stop_words=stop_words, max_features=40000)
text_sample = reindexed_data

document_term_matrix = count_vectorizer.fit_transform(text_sample)



In [ ]:
# Define some required functions

def get_keys(topic_matrix):
    '''
    returns an integer list of predicted topic 
    categories for a given topic matrix
    '''
    keys = topic_matrix.argmax(axis=1).tolist()
    return keys

def keys_to_counts(keys):
    '''
    returns a tuple of topic categories and their 
    accompanying magnitudes for a given list of keys
    '''
    count_pairs = Counter(keys).items()
    categories = [pair[0] for pair in count_pairs]
    counts = [pair[1] for pair in count_pairs]
    return (categories, counts)

def get_top_n_words(n, keys, document_term_matrix, count_vectorizer):
    '''
    returns a list of n_topic strings, where each string contains the n most common 
    words in a predicted category, in order
    '''
    top_word_indices = []
    for topic in range(n_topics):
        temp_vector_sum = 0
        #print(type(temp_vector_sum)) 
        for i in range(len(keys)):
            if keys[i] == topic:
                temp_vector_sum += document_term_matrix[i]
                
        #print(type(temp_vector_sum))     
        temp_vector_sum = temp_vector_sum.toarray()
        #print(type(temp_vector_sum))
        top_n_word_indices = np.flip(np.argsort(temp_vector_sum)[0][-n:],0)
        top_word_indices.append(top_n_word_indices)   
    top_words = []
    for topic in top_word_indices:
        topic_words = []
        for index in topic:
            temp_word_vector = np.zeros((1,document_term_matrix.shape[1]))
            temp_word_vector[:,index] = 1
            the_word = count_vectorizer.inverse_transform(temp_word_vector)[0][0]
            topic_words.append(the_word.encode('ascii','ignore').decode('utf-8'))
        top_words.append(" ".join(topic_words))         
    return top_words


def get_mean_topic_vectors(keys, two_dim_vectors):
    '''
    returns a list of centroid vectors from each predicted topic category
    '''
    mean_topic_vectors = []
    for t in range(n_topics):
        articles_in_that_topic = []
        for i in range(len(keys)):
            if keys[i] == t:
                articles_in_that_topic.append(two_dim_vectors[i])    
        
        articles_in_that_topic = np.vstack(articles_in_that_topic)
        mean_article_in_that_topic = np.mean(articles_in_that_topic, axis=0)
        mean_topic_vectors.append(mean_article_in_that_topic)
    return mean_topic_vectors



### Latent Dirichilet Allocation (LDA)


In [ ]:
n_topics=10
lda_model = LatentDirichletAllocation(n_components=n_topics, learning_method='online', 
                                          random_state=0, verbose=0)
lda_topic_matrix = lda_model.fit_transform(document_term_matrix)

In [ ]:
lda_topic_matrix

Once again, we take the $\arg \max$ of each entry in the topic matrix to obtain the predicted topic category for each headline. These topic categories can then be characterised by their most frequent words.


In [ ]:
lda_keys = get_keys(lda_topic_matrix)
lda_categories, lda_counts = keys_to_counts(lda_keys)

In [ ]:
top_n_words_lda = get_top_n_words(10, lda_keys, document_term_matrix, count_vectorizer)

for i in range(len(top_n_words_lda)):
    print("Topic {}: ".format(i+1), top_n_words_lda[i])

The relative topic compositions of the sample are then illustated with a barchart.

In [ ]:
top_3_words = get_top_n_words(3, lda_keys, document_term_matrix, count_vectorizer)
labels = ['Topic {}: \n'.format(i) + top_3_words[i] for i in lda_categories]

fig, ax = plt.subplots(figsize=(16,8))
ax.bar(lda_categories, lda_counts);
ax.set_xticks(lda_categories);
ax.set_xticklabels(labels);
ax.set_title('LDA topic counts');
ax.set_ylabel('Number of articles');

### Projecting the data into two dimensions for visualization using $t$-SNE.

In [ ]:
tsne_lda_model = TSNE(n_components=2, perplexity=50, learning_rate=100, 
                        n_iter=2000, verbose=1, random_state=0, angle=0.75)
tsne_lda_vectors = tsne_lda_model.fit_transform(lda_topic_matrix)

In [ ]:
colormap = np.array([
    "#1f77b4", "#aec7e8", "#ff7f0e", "#ffbb78", "#2ca02c",
    "#98df8a", "#d62728", "#ff9896", "#9467bd", "#c5b0d5",
    "#8c564b", "#c49c94", "#e377c2", "#f7b6d2", "#7f7f7f",
    "#c7c7c7", "#bcbd22", "#dbdb8d", "#17becf", "#9edae5" ])
colormap = colormap[:n_topics]

In [ ]:
top_3_words_lda = get_top_n_words(3, lda_keys, document_term_matrix, count_vectorizer)
lda_mean_topic_vectors = get_mean_topic_vectors(lda_keys, tsne_lda_vectors)

plot = figure(title="t-SNE Clustering of {} LDA Topics".format(n_topics), plot_width=700, plot_height=700)
plot.scatter(x=tsne_lda_vectors[:,0], y=tsne_lda_vectors[:,1], color=colormap[lda_keys])

for t in range(n_topics):
    label = Label(x=lda_mean_topic_vectors[t][0], y=lda_mean_topic_vectors[t][1], 
                  text=top_3_words_lda[t], text_color=colormap[t])
    plot.add_layout(label)

show(plot)

### Dataset Dataframe 

In [ ]:
topics_docs=[]
docs_percentage=[]
documentid=[]
for i in range(len(lda_topic_matrix)):
    m= np.argmax(lda_topic_matrix[i])
    topics_docs.append(m)
    docs_percentage.append(lda_topic_matrix[i][m])
    documentid.append(i)
td=pd.DataFrame()
td["Article ID"]=documentid
td["Topic"]=topics_docs
td["Percentage Contribution"]=docs_percentage

In [ ]:
td

### Dataset Dataframe (Topic-wise)

In [ ]:
#Separating the articles into lists according to the dominant topic of the article

CORPUS=[]
topic0=[]
topic0per=[]
doc0link=[]
newlist=reindexed_data.index
for i in range(len(newlist)):
    if topics_docs[i]==0:
        topic0.append(i)
        topic0per.append(docs_percentage[i])
        doc0link.append(newlist[i])
t0=pd.DataFrame()
t0['Document ID']=topic0
t0['Percentage Contribution']=topic0per
t0['Topic']=0
t0["Link"]=doc0link
t0.sort_values(by=['Percentage Contribution'], inplace=True,ascending=False)
CORPUS.append(t0)

topic1=[]
topic1per=[]
doc1link=[]
for i in range(len(newlist)):
    if topics_docs[i]==1:
        topic1.append(i)
        topic1per.append(docs_percentage[i])
        doc1link.append(newlist[i])
t1=pd.DataFrame()
t1['Document ID']=topic1
t1['Percentage Contribution']=topic1per
t1['Topic']=1
t1["Link"]=doc1link
t1.sort_values(by=['Percentage Contribution'], inplace=True,ascending=False)
CORPUS.append(t1)

topic2=[]
topic2per=[]
doc2link=[]
for i in range(len(newlist)):
    if topics_docs[i]==2:
        topic2.append(i)
        topic2per.append(docs_percentage[i])
        doc2link.append(newlist[i])
t2=pd.DataFrame()
t2['Document ID']=topic2
t2['Percentage Contribution']=topic2per
t2['Topic']=2
t2["Link"]=doc2link
t2.sort_values(by=['Percentage Contribution'], inplace=True,ascending=False)
CORPUS.append(t2)

topic3=[]
topic3per=[]
doc3link=[]
for i in range(len(newlist)):
    if topics_docs[i]==3:
        topic3.append(i)
        topic3per.append(docs_percentage[i])
        doc3link.append(newlist[i])
t3=pd.DataFrame()
t3['Document ID']=topic3
t3['Percentage Contribution']=topic3per
t3['Topic']=3
t3["Link"]=doc3link
t3.sort_values(by=['Percentage Contribution'], inplace=True,ascending=False)
CORPUS.append(t3)

topic4=[]
topic4per=[]
doc4link=[]
for i in range(len(newlist)):
    if topics_docs[i]==4:
        topic4.append(i)
        topic4per.append(docs_percentage[i]) 
        doc4link.append(newlist[i])
t4=pd.DataFrame()
t4['Document ID']=topic4
t4['Percentage Contribution']=topic4per
t4['Topic']=4
t4["Link"]=doc4link
t4.sort_values(by=['Percentage Contribution'], inplace=True,ascending=False)
CORPUS.append(t4)

topic5=[]
topic5per=[]
doc5link=[]
for i in range(len(newlist)):
    if topics_docs[i]==5:
        topic5.append(i)
        topic5per.append(docs_percentage[i])  
        doc5link.append(newlist[i])
t5=pd.DataFrame()
t5['Document ID']=topic5
t5['Percentage Contribution']=topic5per
t5['Topic']=5
t5["Link"]=doc5link
t5.sort_values(by=['Percentage Contribution'], inplace=True,ascending=False)
CORPUS.append(t5)


topic6=[]
topic6per=[]
doc6link=[]
for i in range(len(newlist)):
    if topics_docs[i]==6:
        topic6.append(i)
        topic6per.append(docs_percentage[i]) 
        doc6link.append(newlist[i])
t6=pd.DataFrame()
t6['Document ID']=topic6
t6['Percentage Contribution']=topic6per
t6['Topic']=6
t6["Link"]=doc6link
t6.sort_values(by=['Percentage Contribution'], inplace=True,ascending=False)
CORPUS.append(t6)

topic7=[]
topic7per=[]
doc7link=[]
for i in range(len(newlist)):
    if topics_docs[i]==7:
        topic7.append(i)
        topic7per.append(docs_percentage[i])   
        doc7link.append(newlist[i])
t7=pd.DataFrame()
t7['Document ID']=topic7
t7['Percentage Contribution']=topic7per
t7['Topic']=7
t7["Link"]=doc7link
t7.sort_values(by=['Percentage Contribution'], inplace=True,ascending=False)
CORPUS.append(t7)

topic8=[]
topic8per=[]
doc8link=[]
for i in range(len(newlist)):
    if topics_docs[i]==8:
        topic8.append(i)
        topic8per.append(docs_percentage[i])  
        doc8link.append(newlist[i])
t8=pd.DataFrame()
t8['Document ID']=topic8
t8['Percentage Contribution']=topic8per
t8['Topic']=8
t8["Link"]=doc8link
t8.sort_values(by=['Percentage Contribution'], inplace=True,ascending=False)
CORPUS.append(t8)

topic9=[]
topic9per=[]
doc9link=[]
for i in range(len(newlist)):
    if topics_docs[i]==9:
        topic9.append(i)
        topic9per.append(docs_percentage[i])    
        doc9link.append(newlist[i])
t9=pd.DataFrame()
t9['Document ID']=topic9
t9['Percentage Contribution']=topic9per
t9['Topic']=9
t9["Link"]=doc9link
t9.sort_values(by=['Percentage Contribution'], inplace=True,ascending=False)
CORPUS.append(t9)

CORPUS[8] #example

### Modeling the time spent reading the articles

In [ ]:
import scipy.stats as ss
import matplotlib.pyplot as plt

avg_time=[]
for i in range(len(article_length)):
    avg_time.append(article_length[i]/250.0)
    
def article_time(article_index):
    n = 1000
    np.random.seed(0x5eed)
    # Parameters of the mixture components
    norm_params = np.array([[avg_time[article_index]/4, avg_time[article_index]/5],
                            [avg_time[article_index],avg_time[article_index]/5]])
    n_components = norm_params.shape[0]
    # Weight of each component, in this case all of them are 1/2
    weights = np.ones(n_components, dtype=np.float64) / float(n_components)
    # A stream of indices from which to choose the component
    mixture_idx = np.random.choice(n_components, size=n, replace=True, p=weights)
    # y is the mixture sample
    y = np.fromiter((ss.norm.rvs(*(norm_params[i])) for i in mixture_idx),
                       dtype=np.float64)
    return(random.choice(y))

    


Below, we show a sample of the time allotment function created above. We're creating the sample points for article at index '1' and plotting them with the expected two component Gaussian.

In [ ]:
n = 10000
np.random.seed(0x5eed)
# Parameters of the mixture components
norm_params = np.array([[avg_time[1]/4, avg_time[1]/5],
                        [avg_time[1],avg_time[1]/5]])
n_components = norm_params.shape[0]
# Weight of each component, in this case all of them are 1/2
weights = np.ones(n_components, dtype=np.float64) / float(n_components)
# A stream of indices from which to choose the component
mixture_idx = np.random.choice(n_components, size=n, replace=True, p=weights)
# y is the mixture sample
y = np.fromiter((ss.norm.rvs(*(norm_params[i])) for i in mixture_idx),
                   dtype=np.float64)

# Theoretical PDF plotting -- generate the x and y plotting positions
xs = np.linspace(y.min(), y.max(), 200)
ys = np.zeros_like(xs)

for (l, s), w in zip(norm_params, weights):
    ys += ss.norm.pdf(xs, loc=l, scale=s) * w

plt.plot(xs, ys)
plt.hist(y, density=True, bins="fd")
plt.xlabel("Time spent reading article")

plt.show()

# CLICKSTREAM GENERATION

Proceed to generate sessions

In [ ]:
CLICKTHROUGH_PER_SESSION_REGULAR_USER=[] #number of articles read by an average user (old)
TIME_SPENT_PER_SESSION_REGULAR_USER=[] #time spent by an average user (old)
CLICKTHROUGH_PER_SESSION_NEW_USER=[] #number of articles read by an average user (new)
TIME_SPENT_PER_SESSION_NEW_USER=[] #time spent by an average user (new)

### First Session

Run this cell to get the profile of new users for the first session 

In [ ]:
new_users=10 #ENTER THE NUMBER OF NEW USERS


NEW_USER_PROFILE=[]
 
for j in range(new_users):
    user_id=[]
    doc_given=[]
    click=[]
    time_per_article=[]
    topic_id=[]
    session_id=[]
    
    
   
    for i in range(10):
        doc_given.append(random.choice(CORPUS[i]['Document ID']))
        topic_id.append(i)
        user_id.append(j)
        session_id.append(1)
        click.append(int(random.getrandbits(1)))
        if click[i]==0:
            time_per_article.append(0)
        else:
            a=1
            while a==1:
                t=article_time(doc_given[i])
                if t>0 :
                    a=0
                    time_per_article.append(t)
    
    profile_new=pd.DataFrame()
    profile_new['User ID']=user_id
    profile_new["Session ID"]=session_id
    profile_new['Document ID']=doc_given
    profile_new['Topic']=topic_id
    profile_new['click']=click
    profile_new['time_spent']=time_per_article

    profile_new.sort_values(by=['User ID'], inplace=True,ascending=True)
    NEW_USER_PROFILE.append(profile_new)

In [ ]:
NEW_USER_PROFILE[0]

Run this cell to generate a session analysis for the new users for the first session

In [ ]:
session_new=pd.DataFrame()
u_id=[]
articles_read=[]
time_spent=[]
session_id=[]
for i in range(new_users):
    u_id.append(i)
    session_id.append(1)
    user_profile=NEW_USER_PROFILE[i]
    time_spent.append(np.sum(user_profile['time_spent']))
    articles_read.append(np.sum(user_profile['click']))
session_new['Session ID']=session_id
session_new['New User ID']=u_id
session_new['Total time spent']=time_spent
session_new['Articles read']=articles_read
session_avg_time=np.sum(session_new['Total time spent']/new_users)
print('Average time spent by a new user in this session',session_avg_time,"\n")
avg_clicks_per_user=np.sum(session_new["Articles read"]/new_users)
print("Average number of articles read by a new user  in this session ", avg_clicks_per_user,"\n")
category=[]
for i in range(new_users):
    if session_new.iloc[i]['Total time spent']<(session_avg_time*1.0/2):
        category.append("Non regular")
    else:
        category.append("Regular")
session_new['Expected category']=category
print(session_new)

### OLD USERS 
User profiling and session analysis for the regular users have been added in a similar manner as above and are contained in the session loop.

### UPDATING THE REGULAR USER REPOSITORY

Herw we add the new regular customers to our existing list of "REGULAR_USERS" and add the new session's data to our existing regular customers

In [ ]:
REGULAR_USERS=[] #0 AT THE BEGINNING

In [ ]:
#UPDATING EXISTING LIST - RUNS FROM THE SECOND SESSION
# for i in range(len(REGULAR_USERS)):
#     REGULAR_USERS[i]=pd.concat([REGULAR_USERS[i],OLD_USER_PROFILE[i]])
 


In [ ]:
#ADDING NEW REGULAR USERS TO THE LIST
n=len(REGULAR_USERS)
print("Number pf regular users before this session",len(REGULAR_USERS))
for i in range(new_users):
    if session_new.iloc[i]['Expected category']=="Regular":
        REGULAR_USERS.append(NEW_USER_PROFILE[i])
        REGULAR_USERS[n].loc[:,"User ID"]=n
        n=n+1
print("\n Number of regular users after this session",len(REGULAR_USERS))

### USER VECTORS OF REGULAR USERS

In [ ]:
user_vector=[]
for i in range(len(REGULAR_USERS)):
    v=np.zeros(10)
    user_vector.append(v)

    user_profile=REGULAR_USERS[i].iloc[0:10]
    total_time=np.sum(user_profile['time_spent'])
    
    for j in range(len(user_profile)):
        user_vector[i]=user_vector[i]+((user_profile.iloc[j]["time_spent"]*1.0/total_time)*lda_topic_matrix[int(user_profile.iloc[j]['Document ID'])])
        

#USES SAME INDEX AS REGULAR_USERS
    

In [ ]:
def cos_sim(a, b):
	"""Takes 2 vectors a, b and returns the cosine similarity according 
	to the definition of the dot product
	"""
	dot_product = np.dot(a, b)
	norm_a = np.linalg.norm(a)
	norm_b = np.linalg.norm(b)
	return dot_product / (norm_a * norm_b)

### SIMILARITY COEFFICIENT FOR RECOMMENDATIONS

In [ ]:
SIMILAR_DOCS=[]
for j in range(len(user_vector)):
    similarity=[]
    docid=[]
    topic_distri=[]
    for i in range(len(lda_topic_matrix)):
        docid.append(i)
        cs= cos_sim(user_vector[j],lda_topic_matrix[i])
        similarity.append(cs)
        topic_distri.append(topics_docs[i])
    #similarity.sort(reverse=True)
    zipped = zip(docid, similarity)
    similar_docs=pd.DataFrame()
    similar_docs['Document ID']=docid
    similar_docs['Similarity']=similarity
    similar_docs['Links']=newlist
    similar_docs["Topic"]=topic_distri
    similar_docs.sort_values(by=['Similarity'], inplace=True,ascending=False)
    SIMILAR_DOCS.append(similar_docs)
print("Top 10 similar documents:")
SIMILAR_DOCS[0].iloc[0:10]

### Further sessions
Run the following looped cell according to the number of sessions to execute.
Please rerun the entire code if you wish to rerun the cell below.

In [ ]:
NUMBER_OF_SESSIONS=2

for m in range(NUMBER_OF_SESSIONS):
    print ("\n SESSION",m+2,"\n")
    new_users=10 #ENTER THE NUMBER OF NEW USERS


    NEW_USER_PROFILE=[]

    for j in range(new_users):
        user_id=[]
        doc_given=[]
        click=[]
        time_per_article=[]
        topic_id=[]
        session_id=[]



        for i in range(10):
            doc_given.append(random.choice(CORPUS[i]['Document ID']))
            topic_id.append(i)
            user_id.append(j)
            session_id.append(m+2)
            click.append(int(random.getrandbits(1)))
            if click[i]==0:
                time_per_article.append(0)
            else:
                a=1
                while a==1:
                    t=article_time(doc_given[i])
                    if t>0 :
                        a=0
                        time_per_article.append(t)

        profile_new=pd.DataFrame()
        profile_new['User ID']=user_id
        profile_new['Session ID']=session_id
        profile_new['Document ID']=doc_given
        profile_new['Topic']=topic_id
        profile_new['click']=click
        profile_new['time_spent']=time_per_article

        profile_new.sort_values(by=['User ID'], inplace=True,ascending=True)
        NEW_USER_PROFILE.append(profile_new)

    session_new=pd.DataFrame()
    u_id=[]
    articles_read=[]
    time_spent=[]
    session_id=[]
    for i in range(new_users):
        u_id.append(i)
        session_id.append(m+2)
        user_profile=NEW_USER_PROFILE[i]
        time_spent.append(np.sum(user_profile['time_spent']))
        articles_read.append(np.sum(user_profile['click']))
    
    session_new['New User ID']=u_id
    session_new['Session ID']=session_id
    session_new['Total time spent']=time_spent
    session_new['Articles read']=articles_read
    session_new_avg_time=np.sum(session_new['Total time spent']/new_users)
    
    print('\n Average time spent by a new user in this session',session_new_avg_time,"\n")
    avg_clicks_per_new_user=np.sum(session_new["Articles read"]/new_users)
    print("\n Average number of articles read by a new user  in this session ", avg_clicks_per_new_user,"\n")
    CLICKTHROUGH_PER_SESSION_NEW_USER.append(avg_clicks_per_new_user)
    TIME_SPENT_PER_SESSION_NEW_USER.append(session_new_avg_time)
    category=[]
    for i in range(new_users):
        if session_new.iloc[i]['Total time spent']<(session_avg_time*1.0/2):
            category.append("Non regular")
        else:
            category.append("Regular")
    session_new['Expected category']=category
    print(session_new)

    
    
    
    ##############################_____________USER PROFILE FOR REGULAR USER__________########################################
   

    
    
    
    OLD_USER_PROFILE=[]


    for j in range(len(REGULAR_USERS)): 

        user_id=[]
        doc_given=[]
        click=[]
        time_per_article=[]
        topic_id=[]
        session_id=[]

        ##RECOMMENDING ARTICLES
        n=0
        ind=0

        while n<5:

            if not(SIMILAR_DOCS[j].iloc[ind]['Document ID'] in REGULAR_USERS[j]["Document ID"]):
                doc_given.append(SIMILAR_DOCS[j].iloc[ind]['Document ID'])
                topic_id.append(SIMILAR_DOCS[j].iloc[ind]['Topic'])
                ind=ind+1
                n=n+1

            else:
                ind=ind+1

        n=0
        while n<5:

            r=np.random.randint(0,len(td))
            random_doc=td.iloc[r]['Article ID']
            if not(random_doc in REGULAR_USERS[j]["Document ID"]) and not(random_doc in doc_given) :
                doc_given.append(random_doc)
                topic_id.append(td.iloc[r]["Topic"])
                n=n+1
        for i in range(10):
            user_id.append(j)
            session_id.append(m+2)
            if SIMILAR_DOCS[j].iloc[int(doc_given[i])]['Similarity']>7:
                a=int(random.getrandbits(1))
                b=int(random.getrandbits(1))
                if a==0 and b==0:
                      click.append(0)
                else:
                      click.append(1)
            else:
                      click.append(int(random.getrandbits(1)))
            if click[i]==0:
                time_per_article.append(0)
            else:
                a=1
                while a==1:
                    t=article_time(int(doc_given[i]))
                    if t>0 :
                        a=0
                        time_per_article.append(t)


        profile_regular=pd.DataFrame()
        profile_regular['User ID']=user_id
        profile_regular["Session ID"]=session_id
        profile_regular['Document ID']=doc_given
        profile_regular['Topic']=topic_id
        profile_regular['click']=click
        profile_regular['time_spent']=time_per_article

        profile_regular.sort_values(by=['User ID'], inplace=True,ascending=True)
        OLD_USER_PROFILE.append(profile_regular)
        
    
    
    #########################_____________SESSION ANALYSIS FOR OLD USERS_____________#####################
    
    
    session_regular=pd.DataFrame()
    u_id=[]
    articles_read=[]
    time_spent=[]
    session_id=[]
    for i in range(len(REGULAR_USERS)):
        u_id.append(i)
        session_id.append(m+2)
        user_profile=OLD_USER_PROFILE[i]
        time_spent.append(np.sum(user_profile['time_spent']))
        articles_read.append(np.sum(user_profile['click']))
    session_regular['User ID']=u_id
    session_regular["Session ID"]=session_id
    session_regular['Total time spent']=time_spent
    session_regular['Number of articles read']=articles_read
    session_regular_avg_time=np.sum(session_regular['Total time spent']/len(REGULAR_USERS))
    print('\n Average time spent by a regular user in this session',session_regular_avg_time,"\n")
    avg_clicks_per_regular_user=np.sum(session_regular["Number of articles read"]/len(REGULAR_USERS))
    print("\n Average number of articles read by a regular user  in this session ", avg_clicks_per_regular_user,"\n")
    print(session_regular)
    
    CLICKTHROUGH_PER_SESSION_REGULAR_USER.append(avg_clicks_per_regular_user)
    TIME_SPENT_PER_SESSION_REGULAR_USER.append(session_regular_avg_time)
    
                                    
                                    
    ####################____________UPDATING REGULARY USER DATASET____________##########################                               
                                    
                                    
    for i in range(len(REGULAR_USERS)):
        REGULAR_USERS[i]=pd.concat([REGULAR_USERS[i],OLD_USER_PROFILE[i]])

    n=len(REGULAR_USERS)
    print("\n Number of regular users before this session",len(REGULAR_USERS),"\n")
    for i in range(new_users):
        if session_new.iloc[i]['Expected category']=="Regular":
            REGULAR_USERS.append(NEW_USER_PROFILE[i])
            REGULAR_USERS[n].loc[:,"User ID"]=n
            n=n+1
    print("\n Number of regular users after this session",len(REGULAR_USERS),"\n")
    
    
    
    
    ################################________USER VECTOR____________##############################
                                    
                                    
    user_vector=[]
    for i in range(len(REGULAR_USERS)):
        v=np.zeros(10)
        user_vector.append(v)

        user_profile=REGULAR_USERS[i].iloc[0:10]
        total_time=np.sum(user_profile['time_spent'])

        for j in range(len(user_profile)):
            user_vector[i]=user_vector[i]+((user_profile.iloc[j]["time_spent"]*1.0/total_time)*lda_topic_matrix[int(user_profile.iloc[j]['Document ID'])])


    ###################################_____SIMILARITY COEFFICIENT________###########################
    
    
    SIMILAR_DOCS=[]
    for j in range(len(user_vector)):
        similarity=[]
        docid=[]
        topic_distri=[]
        for i in range(len(lda_topic_matrix)):
            docid.append(i)
            cs= cos_sim(user_vector[j],lda_topic_matrix[i])
            similarity.append(cs)
            topic_distri.append(topics_docs[i])
        #similarity.sort(reverse=True)
        zipped = zip(docid, similarity)
        similar_docs=pd.DataFrame()
        similar_docs['Document ID']=docid
        similar_docs['Similarity']=similarity
        similar_docs['Links']=newlist
        similar_docs["Topic"]=topic_distri
        similar_docs.sort_values(by=['Similarity'], inplace=True,ascending=False)
        SIMILAR_DOCS.append(similar_docs)
    
    
